In [7]:
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd
import json
import sys
sys.path.append('../')
print(sys.path)
import prompts as p




# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()

DATA_PATH = "../../../Datasets/MAMS-ATSA/Downsampled/test/test_multi_row_few_shot.csv"
OUTPUT_PATH = "../../../Datasets/Evaluations/Sentiment Analysis/dynamic_multi_term_few_shot_pipeline_3.5-turbo.csv"

MODEL = "gpt-3.5-turbo"
TEMP = 0



['/Users/timgutberlet/My Drive (t.gutberlet01@gmail.com)/05 - Coding/06 - Bachelor Thesis/Error-Analysis-with-LLMs/Sentiment Analysis/Pipelines/multi_term', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/timgutberlet/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages', '../', '../']


In [2]:
df = pd.read_csv(DATA_PATH)
prompt_name = "prompt_multi_term_fewshot"
prompt = p.prompt_multi_term_fewshot


df['polarity_pred'] = None
df['prompt_name'] = None
df['prompt'] = None
print(df.head())


   Unnamed: 0  text_id  term_id  \
0           0        0        0   
1           1        0        1   
2           2        0        2   
3           3        0        3   
4           4        1        4   

                                                text  \
0  Amusing details distinguish desserts, from dul...   
1  Amusing details distinguish desserts, from dul...   
2  Amusing details distinguish desserts, from dul...   
3  Amusing details distinguish desserts, from dul...   
4  The server was so busy the night we visited th...   

                          term  polarity  example1_term_id  \
0                     desserts  positive              9500   
1     dulce de leche ice-cream   neutral              9500   
2  chocolate sauce tic-tac-toe   neutral              9500   
3            poached pineapple   neutral              9500   
4                       server  negative              1211   

                                     example1_prompt  example1_sim_score  \
0  

In [3]:
def create_terms(df):
    s = ""
    for term in df['term']:
        s += "'"+ term + "'" + "\n"
    return s

def string_to_json(s):
    start_index = s.find('{')
    end_index = s.rfind('}') + 1
    json_string = s[start_index:end_index]
    json_data = json.loads(json_string)
    return json_data

In [4]:
def execute(df, prompt, prompt_name, start = 0):
    current_text = -1
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, timeout=10)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, timeout=10)
    for i in range(start, len(df)):
        if df["text_id"][i] == current_text:
            continue
        current_text = df["text_id"][i]
        if(df["polarity_pred"][i] != None and df["polarity_pred"][i] != "error"):
            continue
        print(i)
        chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
        input_text = df["text"][i]
        terms_df = df[df["text_id"] == current_text]
        terms = create_terms(terms_df)
        example_prompt_1 = df['example1_prompt'][i]
        example_prompt_2 = df['example2_prompt'][i]
        example_prompt_3 = df['example3_prompt'][i]
        try: 
            result = chain.run(example_1 = example_prompt_1, example_2 = example_prompt_2, example_3 = example_prompt_3, input_text = input_text, terms = terms, callbacks=[handler])
        except Exception as e:
            print(e)
            df.loc[i, 'polarity_pred'] =  'error'
            df.loc[i, 'prompt_name'] = prompt_name
            df.loc[i, 'prompt'] = chain.prompt.format_prompt(example_1 = example_prompt_1, example_2 = example_prompt_2, example_3 = example_prompt_3, input_text = input_text, terms = terms).text
            continue
        handler.langfuse.flush()
        prompt_text = chain.prompt.format_prompt(example_1 = example_prompt_1, example_2 = example_prompt_2, example_3 = example_prompt_3, input_text = input_text, terms = terms).text
        if (i < 5):
            print(prompt_text)
            print("\n")
            print(result)
            print("\n")
        
        if (i % 50 == 0):
            print(str(i) + " of  " + str(len(df)))

        data = string_to_json(result)
        term_sentiments= data["term_sentiments"]
        x = 0
        for j in terms_df["term_id"]:
            if df["term"][j].lower() == term_sentiments[x][0].lower():
                df.loc[j, 'polarity_pred'] =  term_sentiments[x][1]
                df.loc[j, 'prompt_name'] = prompt_name
                df.loc[j, 'prompt'] = prompt_text
            else:
                print(df["term"][j].lower())
                print(term_sentiments[x][0].lower())
                print(prompt_text)
                print("\n")
                print(result)
                print("\n")
                df.loc[j, 'polarity_pred'] =  term_sentiments[x][1]
                df.loc[j, 'prompt_name'] = prompt_name
                df.loc[j, 'prompt'] = prompt_text
            x+=1
    return df

df = execute(df, prompt, prompt_name)
    



0


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


Example 1:
Input: "Food is mediocre, except for Mushroom Custard and Foie Gras in Fire Ice."
Terms: 
'Food'
'Mushroom Custard'

Output: {"term_sentiments": [["Food", "negative"], ["Mushroom Custard", "positive"]]}


Example 2:
Input: "It was the WORST Sangria ever, and their house drink has just a shot of Malibu rum, and at a price of $9 you want more than a shot of a 40% alcohol."
Terms: 
'Sangria'
'price'

Output: {"term_sentiments": [["Sangria", "negative"], ["price", "neutral"]]}


Example 3
Input: "The overpriced ice cream is delicious but the service is atrocious - not only do I have to wait to be admitted into this ice cream joint, but the waiter is a snide, gruff man more suited to be a trucker than an ice cream shop server."
Terms: 
'service'
'cream joint'
'cream shop server'

Output: {"term_sentiments": [["service", "negative"], ["cream joint", "neutral"], ["cream shop server", "neutral"]]}

    
Task:
Input: Amusing details distinguish desserts, from dulce de leche ice-cream

In [5]:
error_rows = df[df['polarity_pred'] == 'error']
print(error_rows)


Empty DataFrame
Columns: [Unnamed: 0, text_id, term_id, text, term, polarity, example1_term_id, example1_prompt, example1_sim_score, example2_term_id, example2_prompt, example2_sim_score, example3_term_id, example3_prompt, example3_sim_score, polarity_pred, prompt_name, prompt]
Index: []


In [6]:
df.head()

,Unnamed: 0,text_id,term_id,text,term,polarity,example1_term_id,example1_prompt,example1_sim_score,example2_term_id,example2_prompt,example2_sim_score,example3_term_id,example3_prompt,example3_sim_score,polarity_pred,prompt_name,prompt
0,0,0,0,"Amusing details distinguish desserts, from dul...",desserts,positive,9500,"Input: ""Food is mediocre, except for Mushroom ...",0.152772,5385,"Input: ""It was the WORST Sangria ever, and the...",0.162247,1320,"Input: ""The overpriced ice cream is delicious ...",0.166906,positive,prompt_multi_term_fewshot,"Example 1:\nInput: ""Food is mediocre, except f..."
1,1,0,1,"Amusing details distinguish desserts, from dul...",dulce de leche ice-cream,neutral,9500,"Input: ""Food is mediocre, except for Mushroom ...",0.152772,5385,"Input: ""It was the WORST Sangria ever, and the...",0.162247,1320,"Input: ""The overpriced ice cream is delicious ...",0.166906,positive,prompt_multi_term_fewshot,"Example 1:\nInput: ""Food is mediocre, except f..."
2,2,0,2,"Amusing details distinguish desserts, from dul...",chocolate sauce tic-tac-toe,neutral,9500,"Input: ""Food is mediocre, except for Mushroom ...",0.152772,5385,"Input: ""It was the WORST Sangria ever, and the...",0.162247,1320,"Input: ""The overpriced ice cream is delicious ...",0.166906,positive,prompt_multi_term_fewshot,"Example 1:\nInput: ""Food is mediocre, except f..."
3,3,0,3,"Amusing details distinguish desserts, from dul...",poached pineapple,neutral,9500,"Input: ""Food is mediocre, except for Mushroom ...",0.152772,5385,"Input: ""It was the WORST Sangria ever, and the...",0.162247,1320,"Input: ""The overpriced ice cream is delicious ...",0.166906,positive,prompt_multi_term_fewshot,"Example 1:\nInput: ""Food is mediocre, except f..."
4,4,1,4,The server was so busy the night we visited th...,server,negative,1211,"Input: ""The server also forgot about our desse...",0.354028,8639,"Input: ""the 3rd however, the waiter forgot to ...",0.410810,10792,"Input: ""We waited an hour for our appetizers, ...",0.416744,negative,prompt_multi_term_fewshot,"Example 1:\nInput: ""The server also forgot abo..."


In [43]:
df.to_csv(OUTPUT_PATH, index = True)